In [1]:
# dictionary of parameters, many of which are not needed in continuous position or discrete momentum simulations
function return_params(;x_step = 0.04, t_step = 0.015, 
                        lower_bdry = -20.0, upper_bdry = 20.0, 
                        t_propagate = 10, t_load = 3pi, n_bloch = 1, 
                        n_ramp = 0.2, n_lat_dep = 0.8, dv = 0.05,
                        n_reduce_x = 1, n_reduce_t = 1, ω_0 = 0, 
                        k_max = 6, l_max = 30, 
                        Δx = 0, ϕ = 0) # default values of paramters
    
    grav_ramp = 0.8576;
    ramp_rate = grav_ramp * n_ramp;
    bloch_period = 8/ramp_rate;
    
    t_bloch = n_bloch * bloch_period;
    t_total = 2*t_load + t_bloch + t_propagate;
    dx = 1/(2*dv);
    xs = lower_bdry*dx : x_step : upper_bdry*dx;
    ts = 0 : t_step : t_total;
    
    n_k = 2k_max + 1;  # for discrete momentum basis
    n_l = 2l_max + 1;  # for discrete position basis
    
    # used to match conditions of continuous position and discrete position bases;
    # each lattice site is of length π
    lower_l_max = Int(floor(lower_bdry*dx/pi));
    upper_l_max = Int(ceil(upper_bdry*dx/pi));
    n_l_match = upper_l_max - lower_l_max + 1;
    
    params = Dict("x_step" => x_step,
                  "t_step" => t_step,
                  "lower_bdry" => lower_bdry, 
                  "upper_bdry" => upper_bdry,
                  "t_propagate" => t_propagate, 
                  "t_load" => t_load, 
                  "n_bloch" => n_bloch, 
                  "t_total" => t_total,
                  "n_ramp" => n_ramp, 
                  "n_lat_dep" => n_lat_dep, 
                  "grav_ramp" => grav_ramp, 
                  "ramp_rate" => ramp_rate, 
                  "bloch_period" => bloch_period,
                  "t_bloch" => t_bloch, 
                  "dv" => dv, 
                  "dx" => dx, 
                  "xs" => xs,
                  "ts" => ts,
                  "n_reduce_x" => n_reduce_x, 
                  "n_reduce_t" => n_reduce_t,
                  "ω_0" => ω_0,
                  "k_max" => k_max,
                  "n_k" => n_k,
                  "l_max" => l_max,
                  "n_l" => n_l,
                  "ϕ" => ϕ,
                  "Δx" => Δx,
                  "lower_l_max" => lower_l_max,
                  "upper_l_max" => upper_l_max,
                  "n_l_match" => n_l_match);
    return params
end

return_params (generic function with 1 method)

In [3]:
# discrete position space things
using Distributions
using DifferentialEquations
using LinearAlgebra
using PyPlot

# hopping element as a function of lattice depth 
function hopping_element(n_lat_dep)
    J = 4/sqrt(pi) * n_lat_dep^(3/4) * exp(-2*sqrt(n_lat_dep));
    return J
end

# Bloch oscillatio Hamiltonian in discrete position basis 
function ham_BO_discrete(params)
    lower_l_max = params["lower_l_max"];
    upper_l_max = params["upper_l_max"];
    n_l = params["n_l_match"];
    ramp_rate = params["ramp_rate"];
    n_lat_dep = params["n_lat_dep"];
    
    l_s = [(pi/4) * l * ramp_rate for l = lower_l_max:upper_l_max];
    J_s = 2*hopping_element(n_lat_dep) * ones(n_l-1);
    ham = SymTridiagonal(l_s, J_s);
    return ham
end

# return the probability density at the l-th lattice site (centered at zeros), in n lattice sites
function ψ0_discrete(l, params)
    Δx = params["Δx"];  # initial position of wavefunction relative to x=0
    σ = params["dx"];   # Heisenberg limited wavefunction, velocity spread σ
    μ = Δx;             # center of initial position of wavefunction
    gaussian = Normal(μ, σ);
    
    # stars and bars! |*|*|*| reminder that each lattice site is of length π
    lower_bound = l * pi - pi/2;
    upper_bound = l * pi + pi/2;
    p = cdf(gaussian, upper_bound) - cdf(gaussian, lower_bound); # probability amplitude
    
    return sqrt(p)
end

# function gaussian_table(x)
#     g_s = [exp(-(x - 2x0)^2) for x0 = -l_max:l_max]; # amplitude at x by gaussians on all sites
#     return g_s
# end

# solver of the Schrodinger equation by 
function solve_SE(params)
    # system of equations with wavefunction separated into real and imaginary parts
    function coupled_SE(du, u, p, t)
        H = ham_BO_discrete(params);
        du[1, :] = H * u[2, :];
        du[2, :] = -1 .* H * u[1, :];
    end
    
    lower_l_max = params["lower_l_max"];
    upper_l_max = params["upper_l_max"];
    n_l = params["n_l_match"];
    t_bloch = params["t_bloch"];
    
    # initial wavefunction where all of imaginary part is zero
    ψ0_real = [ψ0_discrete(l, params) for l = lower_l_max:upper_l_max];
    ψ0_imag = zeros(n_l);
    ψ0 = [ψ0_real'; ψ0_imag'];
    
    tspan = (0.0, t_bloch);
    
    sols = solve(ODEProblem(coupled_SE, ψ0, tspan), reltol=1e-3);
    # the number of solutions is the number of time steps the solver returned
    ts = sols.t;
    nTs = length(sols.t);
    ψf = zeros(n_l, nTs) .+ 0im;
    
    for t = 1:nTs
        sol = sols.u[t];
        # wavefunction at the t-th time step, combining the real and imaginary components
        ψt = [complex(real, imag) for (real, imag) in zip(sol[1,:], sol[2,:])];
        ψf[:, t] = ψt;
    end
    return ψf, ts
end

# localization length vs. latdep in discrete position basis
function plot_localization_length_vs_latdep_by_discrete(lat_deps; n_ramp=0.2, label="")
    localization_lengths = zeros(length(lat_deps));
    for i = 1:length(lat_deps)
        params = return_params(n_lat_dep=lat_deps[i], n_ramp=n_ramp);
        ψf, ts = solve_SE(params);
        
        lower_l_max = params["lower_l_max"];
        upper_l_max = params["upper_l_max"];
        xs = lower_l_max*pi : pi : upper_l_max*pi;
        prob_ψ = (abs.(ψf)).^2;
        localization_lengths[i] = localization_length_calc(prob_ψ, xs, ts);
    end
    PyPlot.plot(lat_deps, localization_lengths, label=label);
end

# no phase vs. latdep in discrete position basis because it all comes out as 0 
# shown by the mathematica simulation from a while ago


# useless plotting stuff!
lat_deps = 4:1:15;
r = 0.2;
# plotting
PyPlot.figure();
PyPlot.title("Localization Length of One Bloch Oscillation vs. Lattice Depth, \nwith and without Tight-Binding Assumption");
PyPlot.ylabel("Localization Length");
PyPlot.xlabel("Lattice Depth");
# plot_localization_length_vs_latdep(lat_deps, n_ramp=r, label="without assumption (continuous)");
plot_localization_length_vs_latdep_by_discrete(lat_deps, n_ramp=r, label="with assumption (discrete)");
PyPlot.legend();
PyPlot.savefig("LL_vs_latdep_diff_methods_stepsize=0.5.pdf");

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1242
┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1242
┌ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1242
┌ Info: Installing matplotlib via the Conda matplotlib package...
└ @ PyCall C:\Users\Richard Parker\.julia\packages\PyCall\ttONZ\src\PyCall.jl:705
┌ Info: Running `conda install -y matplotlib` in root environment
└ @ Conda C:\Users\Richard Parker\.julia\packages\Conda\kLXeC\src\Conda.jl:112


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Richard Parker\.julia\conda\3

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cycler-0.10.0              |   py36h009560c_0          13 KB
    freetype-2.9.1             |       ha9979f8_1         450 KB
    kiwisolver-1.1.0           |   py36ha925a31_0          53 KB
    matplotlib-3.1.1           |   py36hc8f65d3_0         4.9 MB
    pyparsing-2.4.2            |             py_0          61 KB
    pytz-2019.3                |             py_0         231 KB
    ------------------------------------------------------------
                                           Total:         5.7 MB

The following NEW packages will be INSTALLED:

  cycler             pkgs/main/win-64::cycler-0.10.0-py36h009560c_0
  freetype           pkgs/main/win-64::freetype-2.9

UndefVarError: UndefVarError: localization_length_calc not defined

In [ ]:
# discrete momentum space things
using LinearAlgebra
using PyPlot

# Bloch oscillatio Hamiltonian in discrete momentum basis 
function ham_BO(ts, params)
    k_max = params["k_max"];
    n_k = params["n_k"];
    n_lat_dep = params["n_lat_dep"];
    ramp_rate = params["ramp_rate"];
    # make an array of hamiltonians for each t in ts
    Ham = [];
    for t = 1:length(ts)
        kinetic_energy = [0.5(2k - ramp_rate*ts[t]/4)^2 for k = -k_max:k_max];
        Ham = push!(Ham, SymTridiagonal(kinetic_energy, (n_lat_dep./2).*ones(n_k - 1)));
    end
    return Ham
end

# E(t)s for each H(t) for t in ts
function energy_over_time(ts, params)
    # get an array of Hamiltonians for each t in ts
    ham = ham_BO(ts, params);
    # array of eigenvalues/energies for each Hamiltonian of t in ts
    eig_vals = [];
    for t=1:length(ts)
        eig_vals = push!(eig_vals, eigvals(SymTridiagonal(real.(ham[t]))));
    end
    # make an array of arrays into a matrix whose i-th row contains the i-th element the inner arrays prior
    eig_vals = hcat(eig_vals...); 
    return eig_vals
end

# Compute the cumulated phase during Bloch Oscillation by an approximation of the integral ϕ = ∫(-E)dt,
# using a Riemann sum
function phase_calc_by_energy_sum(ts, dt, eig_vals)
    nTs = length(ts);
    first_band = eig_vals[1, :];      # take the first band of energies
    cumulated_phases = zeros(nTs);    # array of cumulated phases over time
    # calculate Riemann sum for ϕ = ∫(-E)dt
    for i=2:nTs
        cumulated_phases[i] = cumulated_phases[i-1] - dt*(first_band[i] + first_band[i-1])/2;
    end
    # return the total phase and all intermediate phases 
    return cumulated_phases[length(cumulated_phases)], cumulated_phases 
end

# phase vs. latdep in discrete momentum basis
function plot_phase_vs_latdep_by_energy_sum(lat_deps; n_ramp=0.2, label="")
    # array of phases as a function of lattice depth
    phases = zeros(length(lat_deps));
    
    for i=1:length(lat_deps)
        params = return_params(n_lat_dep=lat_deps[i], n_ramp=n_ramp);
        t_bloch = params["t_bloch"];
        dt = params["t_step"];
        ts = 0:dt:t_bloch;
        eig_vals = energy_over_time(ts, params);
        phases[i], _ = phase_calc_by_energy_sum(ts, dt, eig_vals);
    end
    # to match the arctan function output
    phases = (phases .% (2pi)) .- pi;
    # plotting
    PyPlot.plot(lat_deps, phases, label=label);
end



# useless graphing stuff!

# graph phase over time
function plot_phase_vs_time(params)
    t_bloch = params["t_bloch"];
    dt = params["t_step"];
    ts = 0:dt:t_bloch;
    eig_vals = energy_over_time(ts, params);
    _, cumulated_phases = phase_calc_by_energy_sum(ts, dt, eig_vals);
    
    # plot phases and phases as would be produced by arctan
    PyPlot.subplot(2, 1, 1);
    PyPlot.title("Cumulated Phase of Wavefunction during One Bloch Period vs. Time Elapsed, \nCalculated by Summation of Energy");
    PyPlot.ylabel("Cumulated Phase");
    PyPlot.xlabel("Time Elapsed");
    PyPlot.plot(ts, cumulated_phases);

    PyPlot.subplot(2, 1, 2)
    PyPlot.title("Cumulated Phase of Wavefunction Modulo Pi during One Bloch Period vs. Time Elapsed, \nCalculated by Summation of Energy");
    PyPlot.ylabel("Cumulated Phase Modulo Pi");
    PyPlot.xlabel("Time Elapsed");
    cumulated_phases_arctan = (cumulated_phases .% (2pi)) .- pi;
    PyPlot.plot(ts, cumulated_phases_arctan);
end

function plot_bandstructure(ts, dt, eig_vals; rows_to_graph=4)
    # graph each energy band if there are more bands than rows_to_graph.
    if size(eig_vals, 1) >= rows_to_graph
        for i = 1:rows_to_graph 
            PyPlot.plot(ts, eig_vals[i, :]);
            
            # annotation test
            t_index = Int(round(30/dt));
            x = ts[t_index];
            y = eig_vals[i, :][t_index];
            PyPlot.plot([x], [y], "ko");
            r_label = @sprintf "(%1.2f, %2.2f)" x y;
            PyPlot.annotate(r_label,
            xy=(x, y),  
            xytext=(x+1, y+0))
        end
    end
end

In [ ]:
# graph cumulated phase after one BO as a function of lattice depth by two evaluation methods:
# discrete momentum simulation, and continuous position simulation

lat_deps = 1:0.01:2;
n_ramp_rate = 0.2;

PyPlot.figure();
title = "Phase of Wavefunction after One Bloch Oscillation vs. Lattice Depth, \nCalculated by Real Space and Momentum Space Methods";
PyPlot.title(title);
PyPlot.ylabel("Phase Angle");
PyPlot.xlabel("Lattice Depth");
plot_phase_vs_latdep_by_energy_sum(lat_deps, n_ramp=n_ramp_rate, label="by discrete momentum");
# plot_phase_vs_latdep(lat_deps, n_ramp=n_ramp_rate, label="by continuous position");
PyPlot.legend();
PyPlot.savefig("phase_vs_lattice_depth_by_different_methods.pdf");

In [ ]:
params = return_params(n_ramp=0.05);
PyPlot.figure(figsize=(10,10));
plot_phase_vs_time(params);
PyPlot.savefig("phase_vs_time_by_energy_sum.pdf");

In [ ]:
using Printf

params = return_params(n_ramp=0.2);
t_bloch = params["t_bloch"];
dt = params["t_step"];
ts = 0:dt:t_bloch;
eig_vals = energy_over_time(ts, params);
# PyPlot.figure();
PyPlot.figure(figsize=(8,5));
plot_bandstructure(ts, dt, eig_vals);
PyPlot.show();